## Step 1 calculate the intermediate attention score w
- It's dot product of the input embedding vector
- The dot product determines the extent to which elements in a sequence attend to each other: the higher the dot product, the higher the similarity and attention score between two elements.
- the score should be normalized so that they sum up to 1


In [2]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

query = inputs[1]  # 2nd input token is the query

attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query) # dot product (transpose not necessary here since they are 1-dim vectors)

print(attn_scores_2)

res = 0.

for idx, element in enumerate(inputs[0]):
    res += inputs[0][idx] * query[idx]

print(res)
print(torch.dot(inputs[0], query))

attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()

print("Attention weights:", attn_weights_2_tmp)
print("Sum:", attn_weights_2_tmp.sum())

# define softmax function
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

attn_weights_2_naive = softmax_naive(attn_scores_2)

print("Attention weights:", attn_weights_2_naive)
print("Sum:", attn_weights_2_naive.sum())

attn_weights_2 = torch.softmax(attn_scores_2, dim=0)

print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())

query = inputs[1] # 2nd input token is the query

context_vec_2 = torch.zeros(query.shape)
for i,x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i]*x_i

print(context_vec_2)

# compute all context vector at a time
attn_scores = torch.empty(6, 6)

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)

print(attn_scores)

attn_scores = inputs @ inputs.T
print(attn_scores)

# normalize
attn_weights = torch.softmax(attn_scores, dim=1)
print(attn_weights)

all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])
tensor(0.9544)
tensor(0.9544)
Attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: tensor(1.0000)
Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)
Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)
tensor([0.4419, 0.6515, 0.5683])
tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])
tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.

## Step2 Implementing self-attention with trainable weights
1. define three trainable weight matrices Wq, Wk, and Wv.
2. compute query (q), key (k), and value (v) vectors for input elements: x ** Wq, x ** Wk, x ** Wv

In [3]:
# 获取输入序列中的第二个元素作为特定的输入向量
x_2 = inputs[1]
# 获取输入张量的嵌入维度，这里假设输入向量的维度为3
d_in = inputs.shape[1]
# 设置输出嵌入的维度，这里假设输出向量的维度为2
d_out = 2

# 设置随机种子以确保结果的可重复性
torch.manual_seed(123)

# 创建查询权重矩阵，形状为 (d_in, d_out)，并且设置 requires_grad=False 表示这些权重在训练过程中不会更新
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

# 创建键权重矩阵，形状和 W_query 相同，同样设置 requires_grad=False
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

# 创建值权重矩阵，形状和 W_query 相同，同样设置 requires_grad=False
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

# 使用查询权重矩阵 W_query 将第二个输入元素 x_2 投影到查询空间
query_2 = x_2 @ W_query  # 使用 @ 运算符进行矩阵乘法

# 使用键权重矩阵 W_key 将第二个输入元素 x_2 投影到键空间
key_2 = x_2 @ W_key

# 使用值权重矩阵 W_value 将第二个输入元素 x_2 投影到值空间
value_2 = x_2 @ W_value

# 使用键权重矩阵 W_key 将输入序列 inputs 投影到键空间
keys = inputs @ W_key

# 使用值权重矩阵 W_value 将输入序列 inputs 投影到值空间
values = inputs @ W_value

# 打印键向量的形状
print("keys.shape:", keys.shape)

# 打印值向量的形状
print("values.shape:", values.shape)

# 打印计算得到的查询向量
print(query_2)

# 从键张量中提取第二个键向量，对应于输入序列中的第二个元素
keys_2 = keys[1]  # Python 中的索引是从 0 开始的

# 计算查询向量 query_2 和键向量 keys_2 之间的点积，得到注意力分数
attn_score_22 = query_2.dot(keys_2)

# 打印计算得到的注意力分数
print(attn_score_22)

# 使用查询向量 query_2 和所有键向量 keys 的转置（keys.T）进行矩阵乘法，得到所有注意力分数
attn_scores_2 = query_2 @ keys.T

# 打印所有注意力分数
print(attn_scores_2)


# 获取键向量的维度，即每个键向量的维度大小
d_k = keys.shape[1]

# 使用 softmax 函数对注意力分数进行归一化处理
# attn_scores_2 / d_k**0.5 是缩放点积分数，防止过大的点积值导致 softmax 梯度过小
# dim=-1 表示沿着最后一个维度（即注意力分数的维度）进行 softmax 计算
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)

# 打印归一化后的注意力权重
print(attn_weights_2)

# 使用归一化的注意力权重 attn_weights_2 和值向量 values 进行矩阵乘法，得到上下文向量
context_vec_2 = attn_weights_2 @ values

# 打印计算得到的上下文向量
print(context_vec_2)

keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])
tensor([0.4306, 1.4551])
tensor(1.8524)
tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])
tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])
tensor([0.3061, 0.8210])


In [4]:
import torch.nn as nn

# 定义自注意力模块
class SelfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out):
        # 调用父类构造函数
        super().__init__()
        # 设置输出维度
        self.d_out = d_out
        # 初始化查询、键和值的权重矩阵，这些矩阵是可训练的
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        # 使用权重矩阵将输入 x 投影到查询、键和值空间
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value
        
        # 计算注意力分数（未归一化）
        attn_scores = queries @ keys.T  # omega
        
        # 使用 softmax 函数和缩放因子归一化注意力分数
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

        # 使用归一化的注意力权重和值向量计算上下文向量
        context_vec = attn_weights @ values
        return context_vec

# 设置随机种子以确保结果的可重复性
torch.manual_seed(123)
# 创建 SelfAttention_v1 实例
sa_v1 = SelfAttention_v1(d_in, d_out)
# 使用输入数据 inputs 进行前向传播，并打印结果
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [5]:
import torch.nn as nn

# 定义自注意力模块的第二个版本
class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        # 调用父类构造函数
        super().__init__()
        # 设置输出维度
        self.d_out = d_out
        # 初始化查询、键和值的线性层，可以选择是否包含偏置项
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        # 使用线性层将输入 x 投影到查询、键和值空间
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        
        # 计算注意力分数（未归一化）
        attn_scores = queries @ keys.T
        
        # 使用 softmax 函数和缩放因子归一化注意力分数
        # 注意这里的 dim=1，表示沿着键向量的维度进行归一化
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=1)

        # 使用归一化的注意力权重和值向量计算上下文向量
        context_vec = attn_weights @ values
        return context_vec

# 设置随机种子以确保结果的可重复性
torch.manual_seed(789)
# 创建 SelfAttention_v2 实例
sa_v2 = SelfAttention_v2(d_in, d_out)
# 使用输入数据 inputs 进行前向传播，并打印结果
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)
